API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [69]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [70]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH


In [71]:
import sqlite3
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import re

In [72]:
####################################
# sqlite 데이터베이스 연결 및 조회(선택)
####################################
sql = "SELECT * FROM PRODUCT"
# SQLite 데이터베이스 연결
conn = sqlite3.connect('../.db/sqlite/my_database.db')
cursor = conn.cursor()

# SQL 쿼리 실행
cursor.execute(sql)
rows = cursor.fetchall()

# 테스트 샘플
print(rows[0])

# 연결 종료
cursor.close()
conn.close()

('0021216860', '엔산 EL-09 이젤형 TV스탠드 티비거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '색상', '화이트 계열', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '176000', '176000', '0', '', '', '176000')


In [5]:
# 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합

# 컬럼 인덱스 정의 (실제 CSV 파일의 컬럼 순서에 맞게 조정해야 합니다)
# 이 부분은 실제 데이터의 컬럼 순서와 일치해야 합니다.
COLUMN_INDICES = {
    "GOODS_NO": 0,
    "GOODS_NM": 1,
    "GOODS_STAT_SCT_NM": 2,
    "BRND_NM": 3,
    "ARTC_INFO": 4,
    "OPT_DISP_NM": 5,
    "OPT_VAL_DESC": 6,
    "CATEGORY_NMS": 7,
    "APPLIANCE_YN": 8,
    "SCH_KWD_NM": 9,
    "SALE_PRC": 10,
    "DSCNT_SALE_PRC": 11,
    "CARD_DC_RATE": 12,
    "CARD_DC_NAME": 13,
    "CARD_DC_NAME_LIST": 14,
    "MAX_BENEFIT_PRICE": 15
}

def generate_search_text_from_tuple(product_data_tuple):
    """
    주어진 상품 정보 튜플에서 검색에 최적화된 통합 텍스트 문자열을 생성합니다.
    머신러닝 기반의 의미론적 검색에 유리하도록 문맥을 구성합니다.

    Args:
        product_data_tuple (tuple): 상품 정보를 담고 있는 튜플.
                                  COLUMNS_INDICES에 정의된 순서와 일치해야 합니다.

    Returns:
        str: 검색 인덱싱을 위한 상품 정보를 결합한 문자열.
    """

    search_components = []

    # 튜플에서 데이터 추출 (COLUMN_INDICES를 활용하여 가독성을 높입니다)
    goods_nm = product_data_tuple[COLUMN_INDICES["GOODS_NM"]].strip()
    goods_stat_sct_nm = product_data_tuple[COLUMN_INDICES["GOODS_STAT_SCT_NM"]].strip()
    brnd_nm = product_data_tuple[COLUMN_INDICES["BRND_NM"]].strip()
    artc_info = product_data_tuple[COLUMN_INDICES["ARTC_INFO"]].strip()
    opt_disp_nm = product_data_tuple[COLUMN_INDICES["OPT_DISP_NM"]].strip()
    opt_val_desc = product_data_tuple[COLUMN_INDICES["OPT_VAL_DESC"]].strip()
    category_nms = product_data_tuple[COLUMN_INDICES["CATEGORY_NMS"]].strip()
    appliance_yn = product_data_tuple[COLUMN_INDICES["APPLIANCE_YN"]].strip()
    sch_kwd_nm = product_data_tuple[COLUMN_INDICES["SCH_KWD_NM"]].strip()
    
    # 숫자 값은 형 변환 시 에러 방지를 위해 get 대신 직접 접근 후 try-except 처리
    sale_prc_str = product_data_tuple[COLUMN_INDICES["SALE_PRC"]]
    dscnt_sale_prc_str = product_data_tuple[COLUMN_INDICES["DSCNT_SALE_PRC"]]
    card_dc_rate_str = product_data_tuple[COLUMN_INDICES["CARD_DC_RATE"]]
    max_benefit_price_str = product_data_tuple[COLUMN_INDICES["MAX_BENEFIT_PRICE"]]
    
    sale_prc = int(sale_prc_str) if sale_prc_str else None
    dscnt_sale_prc = int(dscnt_sale_prc_str) if dscnt_sale_prc_str else None
    card_dc_rate = int(card_dc_rate_str) if card_dc_rate_str else None
    max_benefit_price = int(max_benefit_price_str) if max_benefit_price_str else None

    card_dc_name = product_data_tuple[COLUMN_INDICES["CARD_DC_NAME"]].strip()


    # 1. 브랜드명 + 품목 정보 (카테고리/ARTC_INFO) + 상품명 조합
    # 상품의 핵심적인 정체성을 명확히 합니다.
    if brnd_nm and goods_nm:
        # ARTC_INFO에서 품목의 구체적인 타입을 추론하여 문맥에 더합니다.
        item_type = ''
        if artc_info:
            # '>' 기호로 구분된 품목 정보 중 가장 마지막 부분을 가져와 특정 품목 타입을 얻습니다.
            item_type_parts = [part.strip() for part in artc_info.split('>') if part.strip()]
            if item_type_parts:
                item_type = item_type_parts[-1]
                if '기타' in item_type: # '기타' 라는 모호한 품목이 포함되어 있는 경우
                    if category_nms: # 카테고리의 마지막 부분을 가져옵니다.
                        main_category_parts = [part.strip() for part in category_nms.split('>')]
                        if main_category_parts:
                            item_type = main_category_parts[0]


        # 품목 타입이 있고, 상품명에 이미 품목 타입이 포함되어 있지 않다면 추가합니다.
        if item_type and item_type not in goods_nm:
            search_components.append(f"{brnd_nm}의 {item_type}, {goods_nm}.")
        else: # 품목 타입이 없거나 상품명에 이미 포함되어 있다면 브랜드명과 상품명만 사용합니다.
            search_components.append(f"{brnd_nm}의 {goods_nm}.")
    elif goods_nm: # 브랜드명이 없는 경우 상품명만 사용합니다.
        search_components.append(f"{goods_nm}.")


    # 2. 제품의 주요 특성 (OPT_DISP_NM 및 OPT_VAL_DESC) 조합
    if opt_disp_nm and opt_val_desc:
        display_names = [name.strip() for name in opt_disp_nm.split(',') if name.strip()]
        values = [val.strip() for val in opt_val_desc.split(',') if val.strip()]

        # 각 특성 항목과 값을 '항목: 값' 형태로 결합합니다.
        characteristics = []
        for i in range(min(len(display_names), len(values))):
            # 비어있는 항목이나 '스마트기능'처럼 반복되는 용어는 건너뜁니다.
            # '스마트기능'이 반복되는 경우, 첫 번째 '스마트기능' 뒤에 모든 스마트기능 값을 나열합니다.
            if display_names[i] == '스마트기능':
                if i > 0 and display_names[i-1] == '스마트기능':
                    continue # 이미 처리되었으므로 건너뜁니다.
                
                # 모든 스마트기능 값을 모아서 처리
                all_smart_features = []
                for j in range(i, len(display_names)):
                    if display_names[j] == '스마트기능' and j < len(values):
                        all_smart_features.append(values[j])
                    else:
                        break # 스마트기능이 끝나면 반복 중단

                if all_smart_features:
                    characteristics.append(f"스마트기능: {', '.join(all_smart_features)}")
            else: # 일반적인 특성
                if display_names[i] and values[i]:
                    characteristics.append(f"{display_names[i]}: {values[i]}")

        if characteristics:
            search_components.append("주요 특징: " + ", ".join(characteristics) + ".")

    # 3. 카테고리명 추가
    if category_nms:
        # '|' 기호를 ', '로, '>' 기호를 ' > '로 바꾸어 카테고리 계층 및 다중 카테고리를 표현합니다.
        formatted_categories = category_nms.replace('|', ', ').replace('>', ' > ')
        search_components.append(f"카테고리: {formatted_categories}.")

    # 4. 상품 상태 추가
    if goods_stat_sct_nm and goods_stat_sct_nm != '정상상품': # '정상상품'이 아닌 경우에만 추가합니다.
        search_components.append(f"상품 상태: {goods_stat_sct_nm}.")

    # 5. 가격 및 할인 정보 추가
    price_info_components = []
    if sale_prc is not None:
        price_info_components.append(f"판매가 {sale_prc:,}원") # 천단위 콤마 추가
    if dscnt_sale_prc is not None and sale_prc is not None and dscnt_sale_prc < sale_prc:
        price_info_components.append(f"할인가 {dscnt_sale_prc:,}원") # 천단위 콤마 추가
    if card_dc_rate is not None and card_dc_rate > 0 and card_dc_name:
        if max_benefit_price is not None:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인되어 최대 {max_benefit_price:,}원") # 천단위 콤마 추가
        else:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인")

    if price_info_components:
        search_components.append("가격 정보: " + ", ".join(price_info_components) + ".")

    # 6. 가전제품 여부 추가
    if appliance_yn == 'Y':
        search_components.append("가전제품입니다.")
    else:
        search_components.append("가전제품이 아닙니다.")

    # 7. 해시태그 추가
    if sch_kwd_nm: 
        search_components.append(f"해시태그: {sch_kwd_nm}.")
    
    # 모든 구성 요소를 하나의 문자열로 결합하고 앞뒤 공백을 제거합니다.
    return " ".join(search_components).strip()

In [39]:
# documents 리스트 생성
documents = []
metadatas = []
ids = []


"""
- 예시: 삼성전자의 UHD TV, 125.7cm 4K UHD 비즈니스 TV LH50BEAHLGFXKR (설치유형 선택가능). 카테고리: TV·영상가전 > TV. 가격 정보: 판매가 559,000원.
"""
for row in rows:
    GOODS_NO, GOODS_NM, GOODS_STAT_SCT_NM, BRND_NM, ARTC_INFO, OPT_DISP_NM, OPT_VAL_DESC, CATEGORY_NMS, APPLIANCE_YN, SCH_KWD_NM, SALE_PRC, DSCNT_SALE_PRC, CARD_DC_RATE, CARD_DC_NAME, CARD_DC_NAME_LIST, MAX_BENEFIT_PRICE = row
    metadatas.append(
        {"GOODS_NO": GOODS_NO, 
         "GOODS_NM": GOODS_NM, 
         "GOODS_STAT_SCT_NM": GOODS_STAT_SCT_NM,
         "BRND_NM": BRND_NM,
         "ARTC_INFO": ARTC_INFO,
         "OPT_DISP_NM": OPT_DISP_NM,
         "OPT_VAL_DESC": OPT_VAL_DESC,
         "CATEGORY_NMS": CATEGORY_NMS,
         "APPLIANCE_YN": APPLIANCE_YN,
         "SCH_KWD_NM": SCH_KWD_NM,
         "SALE_PRC": int(SALE_PRC),
         "DSCNT_SALE_PRC": int(DSCNT_SALE_PRC),
         "CARD_DC_RATE": int(CARD_DC_RATE),
         "CARD_DC_NAME": CARD_DC_NAME,
         "CARD_DC_NAME_LIST": CARD_DC_NAME_LIST,
         "MAX_BENEFIT_PRICE": int(MAX_BENEFIT_PRICE)
         })
    search_text = generate_search_text_from_tuple(row)
    documents.append(search_text)
    ids.append(str(GOODS_NO))

for i, meta in enumerate(metadatas[:10]):
    print(meta)

{'GOODS_NO': '0021216860', 'GOODS_NM': '엔산 EL-09 이젤형 TV스탠드 티비거치대', 'GOODS_STAT_SCT_NM': '정상상품', 'BRND_NM': '엔산마운트', 'ARTC_INFO': 'A/V 기타>TV 기타', 'OPT_DISP_NM': '색상', 'OPT_VAL_DESC': '화이트 계열', 'CATEGORY_NMS': 'TV·영상가전>TV>TV액세서리', 'APPLIANCE_YN': 'N', 'SCH_KWD_NM': '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', 'SALE_PRC': 176000, 'DSCNT_SALE_PRC': 176000, 'CARD_DC_RATE': 0, 'CARD_DC_NAME': '', 'CARD_DC_NAME_LIST': '', 'MAX_BENEFIT_PRICE': 176000}
{'GOODS_NO': '0026595956', 'GOODS_NM': '엔산 무드조명형 모니터암 블룸스탠드 삼텐바이미 거치대', 'GOODS_STAT_SCT_NM': '정상상품', 'BRND_NM': '엔산마운트', 'ARTC_INFO': 'A/V 기타>TV 기타', 'OPT_DISP_NM': '', 'OPT_VAL_DESC': '', 'CATEGORY_NMS': 'TV·영상가전>TV>TV액세서리', 'APPLIANCE_YN': 'N', 'SCH_KWD_NM': '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', 'SALE_PRC': 140000, 'DSCNT_SALE_PRC': 140000, 'CARD_DC_RATE': 0, 'CARD_DC_NAME': '', 'CARD_DC_NAME_LIST': '', 'MAX_BENEFIT_PRICE': 140000}
{'GOODS_NO': '0037357623', 'GOODS_NM': '5년무상AS 이스트라 120Hz AI 맥스 127cm UHD 구글 5.0 스마트TV (이젤스탠드A타입-기사)', 'GOODS_STAT_SCT_NM': '정상상품', 'BRND

In [40]:
# 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

In [41]:
# DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

# DB 경로 설정
persist_directory = "../.db/faiss"

# 배치 크기 설정
BATCH_SIZE = 500
total_products = len(documents)

# 첫 번째 배치로 FAISS 초기화
print(f"처리 중: 1~{min(BATCH_SIZE, total_products)}번째 상품 / 총 {total_products}개")
first_batch_docs = documents[:BATCH_SIZE]
first_batch_metadatas = metadatas[:BATCH_SIZE]
first_batch_ids = ids[:BATCH_SIZE]

vectorstore = FAISS.from_texts(
    texts=first_batch_docs,
    embedding=embeddings,
    metadatas=first_batch_metadatas,
    ids=first_batch_ids
)

# 나머지 배치 처리
for i in range(BATCH_SIZE, total_products, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, total_products)
    print(f"처리 중: {i+1}~{end_idx}번째 상품 / 총 {total_products}개")
    
    batch_docs = documents[i:end_idx]
    batch_metadatas = metadatas[i:end_idx]
    batch_ids = ids[i:end_idx]
    
    try:
        vectorstore.add_texts(
            texts=batch_docs,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
    except Exception as e:
        print(f"배치 처리 중 오류 발생: {str(e)}")
        continue

# 로컬에 저장
vectorstore.save_local(persist_directory)

print(f"총 {total_products}개의 상품 정보가 FAISS에 저장되었습니다.")





처리 중: 1~500번째 상품 / 총 3726개
처리 중: 501~1000번째 상품 / 총 3726개
처리 중: 1001~1500번째 상품 / 총 3726개
처리 중: 1501~2000번째 상품 / 총 3726개
처리 중: 2001~2500번째 상품 / 총 3726개
처리 중: 2501~3000번째 상품 / 총 3726개
처리 중: 3001~3500번째 상품 / 총 3726개
처리 중: 3501~3726번째 상품 / 총 3726개
총 3726개의 상품 정보가 FAISS에 저장되었습니다.


In [74]:
# 벡터스토어에서 첫 번째 문서 가져오기
first_doc = vectorstore.similarity_search_with_score(
    query="",  # 빈 쿼리로 검색
    k=1  # 하나의 결과만 가져오기
)[0][0]  # 첫 번째 결과의 문서 객체만 선택

print("\n[벡터스토어 샘플 데이터]")
print("-" * 60)
print(f"문서 내용: {first_doc.page_content}")
print("\n메타데이터:")
for key, value in first_doc.metadata.items():
    print(f"- {key}: {value}")
print("-" * 60)
# 벡터스토어에서 임베딩된 데이터 확인
print("\n[벡터스토어 임베딩 데이터]")
print("-" * 60)
print(f"임베딩 차원: {vectorstore.embedding_function.dimension}")
print(f"저장된 총 문서 수: {vectorstore.index.ntotal}")

# 임베딩 벡터 샘플 확인
sample_vector = vectorstore.index.reconstruct(0)  # 첫 번째 문서의 임베딩 벡터
print(f"\n첫 번째 문서의 임베딩 벡터 (처음 10개 값):")
print(sample_vector[:10])
print("-" * 60)



[벡터스토어 샘플 데이터]
------------------------------------------------------------
문서 내용: 비비텍의 비비텍 단초점 빔프로젝터 MW596ST 빔 프로젝트 학원용. 주요 특징: 방식: DLP, 해상도: HD급, 밝기(안시): 3800안시, 단자: 컴포지트, 단자: 헤드폰, 단자: HDMI, 단자: RGB(D-SUB), 단자: USB, 소음: 32dB, 색상: 화이트 계열. 카테고리: TV·영상가전 > 영상가전 > 빔프로젝터. 가격 정보: 판매가 781,000원. 가전제품입니다. 해시태그: #MW596ST.

메타데이터:
- GOODS_NO: 0018459076
- GOODS_NM: 비비텍 단초점 빔프로젝터 MW596ST 빔 프로젝트 학원용
- GOODS_STAT_SCT_NM: 정상상품
- BRND_NM: 비비텍
- ARTC_INFO: A/V 기타>프로젝터
- OPT_DISP_NM: 방식,해상도,밝기(안시),단자,단자,단자,단자,단자,소음,색상
- OPT_VAL_DESC: DLP,HD급,3800안시,컴포지트,헤드폰,HDMI,RGB(D-SUB),USB,32dB,화이트 계열
- CATEGORY_NMS: TV·영상가전>영상가전>빔프로젝터
- APPLIANCE_YN: Y
- SCH_KWD_NM: #MW596ST
- SALE_PRC: 781000
- DSCNT_SALE_PRC: 781000
- CARD_DC_RATE: 0
- CARD_DC_NAME: 
- CARD_DC_NAME_LIST: 
- MAX_BENEFIT_PRICE: 781000
------------------------------------------------------------

[벡터스토어 임베딩 데이터]
------------------------------------------------------------


AttributeError: 'OpenAIEmbeddings' object has no attribute 'dimension'

In [59]:
def intent_based_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict

In [68]:

# Vectorstore 검색 1 - 검색기만을 통한 검색
vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "현대카드 할인하는 LG전자 TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

# 필터링 옵션 입력 받기
filter_dict = intent_based_filtering(query)
# filter_dict = {"GOODS_NO": '0031055945'}
# filter_dict = {
#     "SALE_PRC": {
#         "$gte": 1000000,  # 최소 가격
#         "$lte": 2000000   # 최대 가격
#     }
# }

print(f"filter_dict: {filter_dict}")

# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    filter = filter_dict, # 메타데이터 필터 적용
    k=5  # 반환할 결과 수
)

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 5개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}] 유사도: {score}") # 낮을수록 더 유사함
        print(f"{doc.page_content}")
        # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
            feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i < 3: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 처리 중 오류 발생: {str(e)}")
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")
    print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")


filter_dict: {}
'현대카드 할인하는 LG전자 TV'와 유사한 상품 검색 결과 (최대 5개):
------------------------------------------------------------
[결과 1] 유사도: 0.7995421290397644
LG전자의 OLED TV, 120cm LG전자 4K UHD TV OLED48C4ENA (설치유형 선택가능). 카테고리: TV·영상가전 > TV > 중형(~139cm). 가격 정보: 판매가 1,490,000원, 롯데카드 카드 사용 시 10% 할인되어 최대 1,341,000원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: 120cm LG전자 4K UHD TV OLED48C4ENA (설치유형 선택가능)
- 품목정보: TV>OLED TV
- 카테고리: TV·영상가전>TV>중형(~139cm)
- 판매가: 1,490,000원
- 할인가: 1,490,000원
- 최대혜택가: 1,341,000원
- 카드할인율: 10%
- 할인카드: 롯데카드,롯데카드,현대카드,네이버페이,삼성카드,신한카드,KB국민카드
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0035753360
------------------------------------------------------------
[결과 2] 유사도: 0.8122871518135071
LG전자의 OLED TV, 194cm 4K UHD evo OLED77C3QNA [설치유형 선택가능]. 카테고리: TV·영상가전 > TV > 초대형(190cm~). 가격 정보: 판매가 6,600,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 6,450,000원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: 194cm 4K UHD evo OLED77C3QNA [설치유형 선택가능]
- 품목정보: TV>OLED TV

In [64]:
####################################
# Vectorstore 검색 2 - LLM을 통한 검색
####################################

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are a product search assistant for an electronics e-commerce platform. 
Use the following pieces of retrieved context to answer the question. 
Search up to 10 products.
Kindly explain why you've been searching for the product.
And be sure to include the following meta information:
상품명
- 상품상태
- 브랜드명
- 판매가
- 할인가
- 최대혜택가
- 할인카드
- 주요 특징 및 기능
  - 
  -
  -
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=상품번호
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [65]:
# Vectorstore 검색 2 - LLM을 통한 검색
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 체인(Chain) 생성
def get_retriever_with_filter(query):
    filter_dict = intent_based_filtering(query)
    return vectorstore.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": filter_dict
        }
    )

chain = (
    {"context": lambda x: get_retriever_with_filter(x).get_relevant_documents(x), 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [77]:
# Vectorstore 검색 2 - LLM을 통한 검색

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
query = "30kg 이상 삼성 스마트 세탁기"  # 예시 쿼리: 원하는 검색어로 변경하세요
response = chain.invoke(query)
print(response)

현재 제공된 문서들에서는 30kg 이상의 삼성 스마트 세탁기에 대한 정보가 없습니다. 제공된 문서들은 주로 18kg에서 25kg 용량의 삼성 세탁기에 대한 정보만 포함하고 있습니다. 

다음은 제공된 문서들에서 찾을 수 있는 삼성 스마트 세탁기 목록입니다:

1. **그랑데 일반세탁기 WA18CG6K46BV 18kg**
   - 상품상태: 정상상품
   - 브랜드명: 삼성전자
   - 판매가: 741,800원
   - 할인가: 667,620원
   - 최대혜택가: 634,240원
   - 할인카드: 롯데카드
   - 주요 특징 및 기능:
     - 세탁 용량: 17~21kg
     - 스마트기능: 코스다운로드, 스마트컨트롤, 스마트체크, 스마트타이머
     - 모터 종류: 인버터 DD모터
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0026697878)

2. **그랑데 일반세탁기 WA18CG6741BW 18kg**
   - 상품상태: 정상상품
   - 브랜드명: 삼성전자
   - 판매가: 1,218,500원
   - 할인가: 1,096,650원
   - 최대혜택가: 1,041,820원
   - 할인카드: 롯데카드
   - 주요 특징 및 기능:
     - 세탁 용량: 18kg
     - 스마트기능: 스마트체크, 스마트타이머
     - 모터 종류: 인버터 DD모터
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0026697880)

3. **삼성 그랑데 통버블 세탁기 18kg WA18CG6K46BV**
   - 상품상태: 정상상품
   - 브랜드명: 삼성전자
   - 판매가: 822,000원
   - 할인가: 748,020원
   - 최대혜택가: 710,620원
   - 할인카드: 롯데카드
   - 주요 특징 및 기능:
     - 세탁 용량: 18kg
     - 스마트기능: 코스다